# Lecture7　回帰分析1
<div dir='rtl'>
2022.4岩政
</div>

## statsmodel
StatsModelsは、統計モデルを用いて推定や検定、探索ができるPythonライブラリです。


ここでは、statsmodelsの最初のIntroductionを紹介します。

【簡単なstatsmodelによる統計解析の例】
https://www.statsmodels.org/stable/index.html


statsmodlesは、R言語のような式によるモデルの定義を行うapiをサポートしています。


statsmodelの組み込みデータを使います。ここではAndre-Michel Guerry (1833) が集めた、1830年付近のパリの社会データ（犯罪、自殺、、、、）のデータ。

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline

dat = sm.datasets.get_rdataset("Guerry", "HistData").data

In [ ]:
dat.head()

86のフランス地方行政区(Department)毎の、
- Lottery: 一人当たりのロイヤル宝くじの掛け金
- Literacy: 識字率
- Pop1831: 人口(1831年調査)
が関係あるかを見ます。プロットしてみます

ここで　Lottery(一人当たりの掛け金)はLiteracy(識識字率)と人口とどう関係するかのモデル式を立てます。'np.log(Pop1831)'とあるのは、Pop1831(1831年の人口)の対数(numpyのlog関数を利用)を予め計算した仮想的な変数='np.log(Pop1831)'を説明変数として用いることを示しています。

式＝'Lottery ~ Literacy + np.log(Pop1831)'


以下は式の文法の一部です。上記式は、くじの掛け金（目的変数）は、識字率と、対数をとった人口の2種類の説明変数の重回帰モデルであることをしめします。

|モデル式|式の意味|
|:---|:---|
|y ~ x| 単回帰モデル、yはxより説明され、切片項がある|
|y ~ x-1|単回帰モデル、"-1"は切片項がないことを意味|
|y ~ x1+x2|重回帰モデル、yはx1とx2により説明、切片項なし|
|y ~ x1:x2| yは交互作用項(x1*x2)で説明される|
|y ~ x1*x2| y ~x1+x2+x1:x2 と同じ

In [ ]:
# Fit regression model (using the natural log of one of the regressors)
results = smf.ols('Lottery ~ Literacy + np.log(Pop1831)', data=dat).fit()

In [ ]:
print(results.summary())

In [ ]:
results.params

つまり回帰式は以下であることを示しています。

y=246.43-0.489*Literacy-np.log(Pop1831)

グラフにしてみます。

In [ ]:
xdata= results.params['Intercept']+ dat['Literacy']+results.params['Literacy']*dat['Literacy']+results.params['np.log(Pop1831)']*np.log(dat['Pop1831'])
plt.scatter(xdata, dat['Lottery'],)
x=np.linspace(50,130,1000)
plt.plot(x,x)

結果の表示

例題は、重回帰（複数の説明変数を使う）回帰なので、説明変数と目的変数の間の関係の可視化は困難です。そこで、説明変数を１つ選択してプロット（部分プロット）をすることができます。

In [ ]:
fig = sm.graphics.plot_partregress_grid(results)
fig.tight_layout(pad=1.0)

部分回帰グラフを表示してみます。

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
ax.title.set_color('white')

plt.style.use('default')
sm.graphics.plot_partregress('Lottery', 'Literacy', ['Pop1831'], ax=ax,
                                data=dat, obs_labels=False);

まとめたもの

In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
# Load data
dat = sm.datasets.get_rdataset("Guerry", "HistData").data
# Fit regression model (using the natural log of one of the regressors)
results = smf.ols('Lottery ~ Literacy + np.log(Pop1831)', data=dat).fit()
# Inspect the results
print(results.summary())

R言語の式以外の書き方（もともとのstatsmodelでのモデル入力のやり方）でやるとこうなります。

ここでは$X$を説明変数、$Y$を目的変数とし

$X=\beta_0+\beta_1x$

とし、

$y=X+\epsilon$

ここで、$\beta$は係数、$\epsilon$はノイズである。

$\beta_0=1,\beta_1=0.1$として人工データを生成しそのデータから$\beta$を推定させる。

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

np.random.seed(9876789)

# 100個のサンプルを生成
nsample = 100
x = np.linspace(0, 10, nsample) # 0から10まで100個等間隔の点列を生成
beta = np.array([1, 0.1]) #人工データでのβ
e = np.random.normal(size=nsample) #ノイズは正規分布から100個生成
X = sm.add_constant(x) # Xは設計行列で、まずはxに定数項を追加
y_true=np.dot(X, beta) # y_trueも確率変数でXと掛け合わせてy_trueを得る
y = y_true + e # y_trueにノイズを付与して人工データyが完成



Xが「設計(design)行列」であることを確認する

In [ ]:
X

データをプロットしてみる

In [ ]:
plt.scatter(x,y)

ここから、最小二乗法(OLS)にて回帰係数を求める

In [ ]:
# Fit regression model
results = sm.OLS(y, X).fit() #データyに対して設計行列(定数とxで構成)で最小2乗回帰
# Inspect the results
print(results.summary())

In [ ]:
results.params

In [ ]:
results.rsquared

回帰結果を、重ねて表示する。wls_prediction_std関数を使って、95%信頼区間も同時に表示

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(results)
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x, y, 'o', label="data")
ax.plot(x, y_true, 'b-', label="True")
ax.plot(x, results.fittedvalues, 'r--.', label="OLS")
ax.plot(x, iv_u, 'r--')
ax.plot(x, iv_l, 'r--')
ax.legend(loc='best');

残差を見てみる

In [ ]:
plt.hist(results.resid)

以上の例を、R-styleでもやってみる




In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import pandas as pd

df=pd.DataFrame({'y':y,'x':x})
df

In [ ]:
# Fit regression model (using the natural log of one of the regressors)
results = smf.ols('y ~ x', data=df).fit()
# Inspect the results
print(results.summary())

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(results)
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(x, y, 'o', label="data")
ax.plot(x, y_true, 'b-', label="True")
ax.plot(x, results.fittedvalues, 'r--.', label="OLS")
ax.plot(x, iv_u, 'r--')
ax.plot(x, iv_l, 'r--')
ax.legend(loc='best');

##### 最尤推定量を求める例
正規分布$N(\mu,\nu)$から標本$X=\{x_1,x_2,\cdots,x_n\}$が得られたとき、平均$\mu$と分散$\nu$の最尤推定量を求める。$\mu,\nu$が与えられた時の確率密度関数$f$は、

$$f(X;\mu,\nu)=\frac{1}{(\sqrt{2\pi\nu})^n}exp\Big(-\frac{\sum_{i=1}^n(x_i-\mu)^2}{2\nu}\Big)$$


$f(X;\mu,\nu)$を$\mu$に対して、プロットしてみます。

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

# データ生成
mu = 0  # 平均
sigma = 1  # 標準偏差
n = 20 # サンプル数
samples = np.random.normal(mu, sigma, n)
print(samples)

# 確率密度関数の計算
mu_vals = np.linspace(-5, 5, 1000)  # μの値を生成
pdf = stats.norm.pdf(samples, loc=mu_vals[:, None], scale=sigma)
pdf_product = np.prod(pdf, axis=1)

# プロット
plt.plot(mu_vals, pdf_product)
#plt.plot(mu_vals, np.log(pdf_product))
plt.xlabel("mu")
plt.ylabel("probability density")
plt.title("Probability density function for mu")
plt.show()


In [ ]:
# logプロット
plt.plot(mu_vals, np.log(pdf_product))
plt.xlabel("mu")
plt.ylabel("probability density")
plt.title("Probability density function for mu")
plt.show()

## 家計調査

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.formula.api as smf

url = 'https://sites.google.com/site/datasciencehiro/datasets/FamilyIncome.csv'
#url = 'datasets/FamilyIncome.csv'
df = pd.read_csv(url, comment='#')
FLAG_fig=True

print(df)

- income: 年間収入
- expendiure: 一か月支出
- engel: エンゲル係数

In [ ]:
df

- ols(ordinary least squares、最小2乗)法を用いて単回帰モデルを求める。
- https://en.wikipedia.org/wiki/Ordinary_least_squares データ数 < 20 の場合には、尖度（kurtosis）の評価（olsではkurtosistestと表記）が有効にできないというwarningメッセージが現れるが、ここではこの評価は使わないので無視する。
- scipy.stats.kurtosistest https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kurtosistest.html
- Kurtosis: https://en.wikipedia.org/wiki/Kurtosis

最初に、一か月支出(expenditure)と年間収入(income)の関係を回帰分析する

In [ ]:
result = smf.ols('expenditure ~ income', data=df).fit()
print(result.summary())

In [ ]:
result.params

In [ ]:
b0, b1 = result.params
prstd, iv_l, iv_u = wls_prediction_std(result)
plt.plot(df['income'],df['expenditure'], 'o')
plt.plot(df['income'], b0+b1*df['income'], c='k')
plt.plot(df['income'], iv_u, 'r--')
plt.plot(df['income'], iv_l, 'r--')
plt.xlabel('income [x 10000YEN]')
plt.ylabel('expenditure')

plt.tight_layout()
if FLAG_fig: plt.savefig('REG_Simple_FamilyIncome1.png')
#plt.show()

予測

In [ ]:
NewData = {'income':[1100,1200]}
df_new = pd.DataFrame(NewData)
pred = result.predict(df_new)
pred

In [ ]:
df_new

上のグラフを見ると，最低収入と最高収入のデータが最も外れているように見えるので，この2点をはずした回帰分析を再度行う。

In [ ]:
df1=df.copy()
df1.drop(9, inplace=True)
df1.drop(0, inplace=True)

In [ ]:
result = smf.ols('expenditure ~ income', data=df1).fit()
result.summary()

In [ ]:
a, b = result.params
prstd, iv_l, iv_u = wls_prediction_std(result)
plt.plot(df1['income'],df1['expenditure'], 'o', c='k')
plt.plot(df1['income'], a+b*df1['income'])
plt.plot(df1['income'], iv_u, 'r--')
plt.plot(df1['income'], iv_l, 'r--')
plt.xlabel('income [x 10000YEN]')
plt.ylabel('expenditure')
plt.show()

95%信頼区間が狭まりました

エンゲル計数と年間収入を回帰分析する

In [ ]:
result = smf.ols('engel ~ income', data=df1).fit()
result.summary()

In [ ]:
a, b = result.params
prstd, iv_l, iv_u = wls_prediction_std(result)
plt.plot(df1['income'],df1['engel'], 'o', c='k')
plt.plot(df1['income'], a+b*df1['income'])
plt.plot(df1['income'], iv_u, 'r--')
plt.plot(df1['income'], iv_l, 'r--')
plt.xlabel('income [x 10000YEN]')
plt.ylabel('engel')
plt.show()

## 多項式回帰分析
Rデータセットのcarsのデータを用いる。
carsの説明 : 次のサイトからcarsを検索  
https://stat.ethz.ch/R-manual/R-＃#devel/library/datasets/html/00Index.html <br>
このデータを予め取得して、下記のように置いた。<br>
これを読み込み、多項式回帰分析を説明する。<br>

本Notebookと類似のREG_Poly_R_cars.ipynbはRデータセットを読込むために、別途、必要なパッケージを予めインストールして、この上で、Rデータセットを読込む。このインストールを省くことを行ったのが本Notebookである。

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.formula.api as smf

FLAG_fig = True

In [ ]:
url = "https://sites.google.com/site/datasciencehiro/datasets/cars_R_datasets.csv"
#url = "datasets/cars_R_datasets.csv"
df = pd.read_csv(url)  # read datasets of cars
x = df.speed
df.head()

#### 1次モデル
$y = b_0 + b_1 x$

In [ ]:
result1 = smf.ols('dist ~ speed', data=df).fit()
print(result1.summary())
b0, b1 = result1.params

In [ ]:
df.plot(kind='scatter', x='speed', y='dist')
plt.plot(x, b0+b1*x)

if FLAG_fig: plt.savefig('fig_REG_poy_R_cars_01.png')

#### 2次モデル
$y = b_0 + b_1 x + b_2 x^2$

In [ ]:
result2 = smf.ols('dist ~ np.power(speed,2) + speed', data=df).fit()
print(result2.summary())
b0, b2, b1 = result2.params

In [ ]:
df.plot(kind='scatter', x='speed', y='dist')
plt.plot(x, b0+b1*x+b2*(x**2))

if FLAG_fig: plt.savefig('fig_REG_poy_R_cars_02.png')

#### 3次モデル
$y = b_0 + b_1  + b_2 x^2 + b_3 x^3$

In [ ]:
result3 = smf.ols('dist ~ np.power(speed,3) + np.power(speed,2) + speed', data=df).fit()
print(result3.summary())
b0, b3, b2, b1 = result3.params

In [ ]:
df.plot(kind='scatter', x='speed', y='dist')
plt.plot(x, b0+b1*x+b2*(x**2) + b3*(x**3))

if FLAG_fig: plt.savefig('fig_REG_poy_R_cars_03.png')

## nupmy.polyfit（）を用いたカーブフィッティングの例

In [ ]:
x = df.speed
y = df.dist
degree = 2
fit = np.polyfit(x, y, degree)
print(fit)
est = np.poly1d(fit)
print(est)
plt.scatter(x,y)
plt.plot(x,est(x))
plt.title('degree = %d' % degree)

In [ ]:
degree = 3
fit = np.polyfit(x, y, degree)
print(fit)
est = np.poly1d(fit)
print(est)
plt.scatter(x,y)
plt.plot(x,est(x))
plt.title('degree = %d' % degree)

overfittingの例

In [ ]:
degree = 9
fit = np.polyfit(x, y, degree)
print(fit)
est = np.poly1d(fit)
print(est)
plt.scatter(x,y)
plt.plot(x,est(x))
plt.title('degree = %d' % degree)